# Cat Face GAN model  

This is a project to test some of the concepts I have learnt for a Generative Adversial Network (GAN) and also to practise using Git. 

## Step 1: Dataset Inspection 

Since the dataset I've obtained from https://github.com/Ferlix/Cat-faces-dataset should already be standardised to 29,842 64x64 RRB images of cats' faces, there shouldn't be a need to normalise them any further. However, it is still good practice to just take a look at the the pics just to make sure. 

In [18]:
import os 
from matplotlib.image import imread
import numpy as np

In [13]:
%cd "D:/data_science/kaggle/CatFaceGAN"
%pwd

D:\data_science\kaggle\CatFaceGAN


'D:\\data_science\\kaggle\\CatFaceGAN'

In [14]:
imgpath=r'dataset'

In [16]:
dimension1 = []
dimension2 = []
colours = []
for filename in os.listdir(imgpath):
    img = imread(imgpath + '\\' + filename)
    d1,d2,colour = img.shape
    dimension1.append(d1)
    dimension2.append(d2)
    colours.append(colour)

In [20]:
print (np.min(dimension1))
print (np.max(dimension1))
print (np.min(dimension2))
print (np.max(dimension2))
print (np.min(colours))
print (np.max(colours))
print(len(dimension1))

64
64
64
64
3
3
29843


As shown above, it is safe to assume that all 29,843 images are 64x64 and are RGB mode. 

## Step 2: creating GAN model

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Dropout, LeakyReLU

In [36]:
coding_size = 100 
#coding size will be the start of the generator shape 
#it should be smaller than the size of the final image (64x64) but not too small

In [48]:
#generator is a model that takes in noise and creates images 
#these images will try to bypass the classifier (discriminator) and get fake images 
generator = Sequential() 
generator.add(Dense(8*8*128, input_shape=[coding_size]))
generator.add(Reshape([8, 8, 128]))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding="same",
                                 activation="relu"))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding="same",
                                 activation="relu"))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding="same",
                                 activation="sigmoid"))

In [49]:
generator.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 8192)              827392    
_________________________________________________________________
reshape_5 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 8, 8, 128)         512       
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 16, 16, 64)        204864    
_________________________________________________________________
batch_normalization_14 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 32, 32, 64)        102464    
_________________________________________________________________
batch_normalization_15 (Batc (None, 32, 32, 64)       

In [50]:
#discriminator is a classification model, employing typical CNN techniques 
discriminator = Sequential() 
discriminator.add(Conv2D(64, kernel_size=5, strides=1, padding="same",
                        activation=LeakyReLU(0.3),
                        input_shape=[64, 64, 3]))
discriminator.add(MaxPooling2D(2,2))
discriminator.add(Dropout(0.5))
discriminator.add(Conv2D(64, kernel_size=5, strides=1, padding="same",
                        activation=LeakyReLU(0.3)))
discriminator.add(MaxPooling2D(2,2))
discriminator.add(Dropout(0.5))
discriminator.add(Conv2D(64, kernel_size=5, strides=2, padding="same",
                        activation=LeakyReLU(0.3)))
discriminator.add(Dropout(0.5))
discriminator.add(Flatten())
discriminator.add(Dense(64, activation='relu'))
discriminator.add(Dense(64, activation='relu'))
discriminator.add(Dense(1, activation="sigmoid"))

In [51]:
discriminator.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 64)        102464    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 8, 64)        

In [52]:
GAN=Sequential([generator,discriminator])

In [53]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.trainable = False

In [59]:
GAN.compile(loss="binary_crossentropy", optimizer="adam")

## Step 3: loading the images into train data 

In [54]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [58]:
datagen = ImageDataGenerator(horizontal_flip=True, rescale=1/255)
train_datagen = datagen.flow_from_directory(directory='D:/data_science/kaggle/CatFaceGAN', target_size=(64,64),color_mode='rgb',
                                            batch_size=256)

Found 29843 images belonging to 1 classes.


## Step 4: training GAN model 